### Beam search

#### Greedy decoding

> - Greedy decoding has no way to undo decisions

#### Exhaustive search

> - Ideally, we want to find a (length $T$) translation $y$ that maximizes
> - $P(y|x) = P(y_1|x) P(y_2|y_1, x) \dots P(y_T|y_1,\dots,y_{T-1}, x) = \prod_1^T P(y_t|y_1, \dots, y_{t-1}, x) $

> - We could try computing **all possible sequences** $y$
    - This means that on each step $t$ of the decoder, we are tracking $V^t$ possible partial translations, where $V$ is the vocabulary size
    - This $O(V_t)$ complexity is far too expensive
    
#### Beam search

> - Core idea: on each time step of the decoder, we keep track of the $k$ most probable partial translations (which we call hypothese)
    - $k$ is the beam size (in practice around 5 to 10)
    
> - A hypothesis $y_1,\dots,y_t$ has a score of its log probability:
    - $\mathrm{score}(y_1, \dots, y_t) = \log{P_{LM}}(y_1, \dots, y_t | x) = \sum_{i=1}^t \log{P_{LM}}(y_i|y_1, \dots, y_{i-1}, x)$
    - Scores are all negative, and a higher score is better
    - We search for high-scoring hypotheses, tracking the top $k$ ones on each step
    
> - Beam search is **not guaranteed** to find a globally optimal solution. 
> - But it is **much more efficient** than exhaustive search!

> #### Beam search: Stopping criterion
> - In **`greedy decoding`**, usually we decode until the model produces a **<END> token**
    - For example: <START> he hit me with a pie <END>
> - In **`beam search decoding`**, different hypotheses may produce <END> tokens on **different timesteps**
    - When a hypothesis produces <END>, that hypothesis is **complete**
    - **Place it aside** and continue exploring other hypotheses via beam search
> - Usually we continue beam search until:
    - We reach timestep $T$ (where $T$ is some pre-defined cutoff), or
    - We have at least $n$ completed hypotheses (where $n$ is the pre-defined cutoff)

> #### Beam search: Finishing up
> - We have our list of completed hypotheses
> - How to select the top one with the highest score?
> - Each hypothesis $y_1, \dots, y_t$ on our list has a score
    - $\mathrm{score}(y_1, \dots, y_t) = \log{P_{LM}}(y_1, \dots, y_t | x) = \sum_{i=1}^t \log{P_{LM}}(y_i|y_1, \dots, y_{i-1}, x)$
> - Problem with this : **longer hypotheses have lower scores**
> - Fix: Normalize by length
    - $\mathrm{score}(y_1, \dots, y_t) = \frac{1}{t} \sum_{i=1}^t \log{P_{LM}}(y_i|y_1, \dots, y_{i-1}, x)$

### BLEU score

> #### Precision and Recall
> - Reference: **Half** of **my heart is in** Havana **ooh na** na
> - Predicted: **Half** as **my heart is in** Obama **ooh na**
> - $ precision = \frac{\#(correct words)}{length \ of \ prediction} = \frac{7}{9} = 78%$
> - $ recall = \frac{\#(correct words)}{length \ of \ reference} = \frac{7}{10} = 70%$
> - $ F-measure = \frac{precision \ * \ recall}{\frac{1}{2}(precision + recall)} = \frac{0.78*0.7}{0.5*(0.78+0.7)} = 73.78%$
> - **Flaw: no penalty for reordering**

> #### BiLingual Evaluation Understudy (BLEU) 
> - **N-gram overlap** between machine translation output and reference sentence
> - Compute **precision for n-grams of size one to four**
> - Add brevity penalty (for too short translations)
    - $BLUE = \min{(1, \frac{length \ of \ prediction}{length \ of \ reference}}) (\prod_{i=1}^4 precision_i)^\frac{1}{4}$
    - Typically computed over the entire corpus, not on single sentences